In [1]:
import os

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import json
import xgboost as xgb
import torch
import shutil
from PIL import Image
from osgeo import gdal
import math
import rasterio
from rasterio.enums import Resampling
from torchvision import models
from torchvision import transforms
from torch import nn

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [4]:
nan_val = -32768.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def pad(vh, rows, cols):
    """
    Pad an image to make it divisible by some block_size.
    Pad on the right and bottom edges so annotations are still usable.
    """
    r, c = vh.shape
    to_rows = math.ceil(r / rows) * rows
    to_cols = math.ceil(c / cols) * cols
    pad_rows = to_rows - r
    pad_cols = to_cols - c
    vh_pad = np.pad(
        vh, pad_width=((0, pad_rows), (0, pad_cols)), mode="constant", constant_values=0
    )
    return vh_pad, pad_rows, pad_cols

def get_grid_coords(padded_img, chips, grids):
    """
    Obtain grid coordinates for chip origins from padded images
    """
    chip_size = chips[0].shape[0]
    grid_coords_y = np.linspace(0, padded_img.shape[0] - chip_size, grids.shape[0])
    grid_coords_x = np.linspace(0, padded_img.shape[1] - chip_size, grids.shape[1])
    grid_coords = [(int(x), int(y)) for y in grid_coords_y for x in grid_coords_x]
    return grid_coords

def scene_pixels_to_chip_pixels(chips, grid_coords, scene_rows, scene_cols):
    """
    Convert scene-level pixel indices for detections to chip-level indices
    """
    chip_size = chips[0].shape[0]
    chip_rows = (scene_rows) % (chip_size)
    chip_cols = (scene_cols) % (chip_size)
    chip_ind_row = (scene_rows) // chip_size * chip_size
    chip_ind_col = (scene_cols) // chip_size * chip_size
    chip_index = [grid_coords.index((c, r)) for c, r in zip(chip_ind_col, chip_ind_row)]
    return chip_rows, chip_cols, chip_index, grid_coords

def chip_sar_img(input_img, sz):
    """
    Takes a raster from xView3 as input and outputs
    a set of chips and the coordinate grid for a
    given chip size
    Args:
        input_img (numpy.array): Input image in np.array form
        sz (int): Size of chip (will be sz x sz x # of channlls)
    Returns:
        images: set of image chips
        images_grid: grid coordinates for each chip
    """
    # The input_img is presumed to already be padded
    images = view_as_blocks(input_img, (sz, sz))
    images_grid = images.reshape(
        int(input_img.shape[0] / sz), int(input_img.shape[1] / sz), sz, sz
    )
    return images, images_grid


def view_as_blocks(arr, block_size):
    """
    Break up an image into blocks and return array.
    """
    m, n = arr.shape
    M, N = block_size
    return arr.reshape(m // M, M, n // N, N).swapaxes(1, 2).reshape(-1, M, N)

def chip_coord_to_scene(row_x, row_y, vh_grid, chip_idx):
    chip_row = chip_idx // vh_grid.shape[1]
    chip_col = chip_idx - (vh_grid.shape[1] * (chip_idx // vh_grid.shape[1]))
    return (chip_row * chip_sz) + row_x, (chip_col * chip_sz) + row_y

In [6]:
%%time
names = ['bay_mean', 'vh_mean', 'vh_max', 'vv_max', 'vh_min', 'vv_min', 'land', 'NaN', 'label', 'scene', 'chip', 'num']
df_gbt = pd.read_csv("train_GBT.csv", names=names[:-3])
df_means = df_gbt.copy()
min_vh = df_means[df_means['vh_min'] != nan_val]['vh_min'].min()
min_vv = df_means[df_means['vv_min'] != nan_val]['vv_min'].min()
min_bay = df_means[(df_means['bay_mean'] != nan_val) & (df_means['NaN'] == 0)]['bay_mean'].min()
mean_vh = df_means[df_means['NaN'] != 1]['vh_mean'].mean()
mean_vv = -15 #df_means[df_means['NaN'] != 1]['vv_mean'].mean()
mean_bay = df_means[df_means['NaN'] != 1]['bay_mean'].mean()
std_vh = 7.4
std_vv = 5.15
std_bay = 1600

CPU times: user 1.92 s, sys: 360 ms, total: 2.27 s
Wall time: 4.5 s


In [2]:
import pandas as pd
p = pd.read_csv("submit_v6.csv")

In [3]:
p.shape

(53575, 6)

In [7]:
class drop_nans(object):
    def __call__(self, image):
        image[0, :, :][image[0, :, :] == -32768.] = min_vh - 10
        image[1, :, :][image[1, :, :] == -32768.] = min_vv - 10
        image[2, :, :][image[2, :, :] == -32768.] = min_bay - 100
        return image

class norm(object):
    def __call__(self, image):
        image[0, :, :] = (image[0, :, :] - mean_vh) / std_vh
        image[1, :, :] = (image[1, :, :] - mean_vv) / std_vv
        image[2, :, :] = (image[2, :, :] - mean_bay) / std_bay
        return image

In [8]:
transform = transforms.Compose(
    [drop_nans(),
    norm()])

### GBT

In [9]:
bst = xgb.Booster()  # init model
bst.load_model('gbt_v1.model')  # load data

### Classification

In [10]:
class_net = models.resnet18()
class_net.fc = nn.Linear(in_features=512, out_features=4, bias=True)
class_net.load_state_dict(torch.load('saved_models/resnet9.pth'))
class_net.to(device)
class_net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Jitter

In [11]:
jitter_net = models.resnet18()
jitter_net.fc = nn.Linear(in_features=512, out_features=2, bias=True)
jitter_net.load_state_dict(torch.load('saved_models/resnet_jitter_9.pth'))
jitter_net.to(device)
jitter_net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Length

In [12]:
length_net = models.resnet18()
length_net.fc = nn.Linear(in_features=512, out_features=1, bias=True)
length_net.load_state_dict(torch.load('saved_models/resnet_length_v2_11.pth'))
length_net.to(device)
length_net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Architecture

- Load up a scene
    -Chip, store chips and labels on disk (!!)
    -Re-chip and (attempt) to store chips and labels on disk (!!)
       -iterate through small chips
           -store means an all vals for scene
           -Run through the GBT and identify candidate scence
           -Put those through first nerual net
               -store class and discard no detection
               -estimate length and jitter coords
               -put in dataframe

In [13]:
chip_sz = 50
base = '/home/oqbrady/data/data/public/'

In [ ]:
predictions = []
data_list = os.listdir('/home/oqbrady/data/data/public')
for scene in tqdm(data_list):
    vh = scene + '/' + 'VH_dB.tif'
    vv = scene + '/' + 'VV_dB.tif'
    bay = scene + '/' + 'bathymetry.tif'
    vh = rasterio.open(base + vh)
    vh = vh.read(1)
    vv = rasterio.open(base + vv)
    vv = vv.read(1)
    bay = rasterio.open(base + bay)
    bay = bay.read(
            out_shape=(
                vh.shape[0],
                vh.shape[1],
            ),
            resampling=Resampling.bilinear,
        ).squeeze()
    vh, _, _ = pad(vh, chip_sz, chip_sz)
    vv, _, _ = pad(vv, chip_sz, chip_sz)
    bay, _, _ = pad(bay, chip_sz, chip_sz)
    vh, vh_grid = chip_sar_img(vh, chip_sz)
    vv, vv_grid = chip_sar_img(vv, chip_sz)
    bay, bay_grid = chip_sar_img(bay, chip_sz)
    bay_mean_tot = []
    vh_mean_tot = []
    vh_max_tot = []
    vv_max_tot = []
    vh_min_tot = []
    vv_min_tot = []
    land_tot = []
    nan_tot = []
    for idx in range(vh.shape[0]):
        bay_mean_tot.append(np.mean(bay[idx]))
        vh_mean_tot.append(np.mean(vh[idx]))
        vh_max_tot.append(np.max(vh[idx]))
        vv_max_tot.append(np.max(vv[idx]))
        vh_min_tot.append(np.min(vh[idx]))
        vv_min_tot.append(np.min(vv[idx]))
        index = np.where(bay[idx] >= 0)
        index1 = np.where(vh[idx] == -32768.0)
        if (len(index[0]) > 0):
            land = 1
        else:
            land = 0
        if (len(index1[0]) > 0):
            nan = 1
        else:
            nan = 0
        land_tot.append(land)
        nan_tot.append(nan)
    data = pd.DataFrame({'bay_mean': bay_mean_tot, 'vh_mean' : vh_mean_tot, 'vh_max' : vh_max_tot, 
                        'vv_max' : vv_max_tot, 'vh_min' : vh_min_tot, 'vv_min' : vv_min_tot, 'land' : land_tot,
                        'NaN' : nan_tot})
    dtest = xgb.DMatrix(data)
    ypred = bst.predict(dtest)
    ypred_bool = ypred > 0.5
    data['preds'] = ypred_bool
    for i in (data[data['preds']]).index:
        image = np.array((vh[i], vv[i], bay[i]))
        image = transform(image)
        image = torch.from_numpy(image).to(device)
        image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
        class_idx = class_net(image)
        pred = torch.argmax(class_idx).item()
        if pred != 0:
            if pred == 1:
                is_vessel = False
                is_fishing = False
            if pred == 2:
                is_vessel = True
                is_fishing = False 
            if pred == 3:
                is_vessel = True
                is_fishing = True
            jitter = jitter_net(image).detach().cpu().numpy()
            row = int(25 - jitter.squeeze()[0])
            col = int(25 - jitter.squeeze()[1])
            scene_row, scene_col = chip_coord_to_scene(row, col, vh_grid, i)
            length = length_net(image).item()
            predictions.append([scene, scene_row, scene_col, is_vessel, is_fishing, length])
    del vh_grid
    del vv_grid
    del bay_grid
    del vh
    del vv
    del bay
submit_csv = pd.DataFrame(predictions, columns=['scene_id', 'detect_scene_row', 'detect_scene_column', 'is_vessel', 
                                             'is_fishing', 'vessel_length_m'])
submit_csv.to_csv("submit_v6.csv", index=False)

  6%|█████▎                                                                                  | 9/150 [13:19<3:27:22, 88.24s/it]